In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import re
import sys
sys.path.append('/home/midzay/Mlerning/bigartm/python')
import artm
import warnings
warnings.filterwarnings('ignore')


from ast import literal_eval
from tqdm import tqdm
from datetime import datetime as time


In [2]:
PATH = 'data/all/' # Путь  к объединенным и обработанным датасетам 
PATH_BIGARTM_MODEL='data/model/bigartm/'

files = list(os.walk(PATH))[0][2]
print(files)

['Internet_i_SMI.csv.bz2', 'Biznes.csv.bz2', 'Sport.csv.bz2']


In [3]:
select_topics = 15

In [4]:


for file in files:
    # загружаем и каждый предобработанный!!!!  датасет из PATH 
    df = pd.read_csv(PATH+file)
    df = df[df.text!='text']
    df.lemmatized_text = df.lemmatized_text.apply(lambda x: literal_eval(x))
    df['year'] = df['date'].apply(lambda x: int(x.split(' ')[0].split('-')[0]))
    df['month'] = df['date'].apply(lambda x: int(x.split(' ')[0].split('-')[1]))
    df = df[df.year>2007]
    df['docID'] = list(range(df.shape[0]))
    df = df[['docID','year','month','lemmatized_text','text']]
    df_data = df[['year','month']]
    
    # Приводим к формату  vowpal_wabbit
    vwpath= f'data/vwpath/{file}_input_bigartm.vw'
    with open(vwpath, 'w') as fp:
        for text, did in df[['lemmatized_text', 'docID']].values:
            fp.write('{} | {}\n'.format(did, ' '.join(text)))
    
    # Создаем batches 
    batches_path = f'data/batches/{file}'
    if not os.path.exists(batches_path):
      
        os.makedirs(batches_path)
    
    batch_vectorizer = artm.BatchVectorizer(data_path=vwpath,data_format='vowpal_wabbit',target_folder=batches_path)
    
    # Создаем словарь
    dictionary = artm.Dictionary()
    dictionary.gather(data_path=batches_path)
    dictionary.filter(min_tf=10, max_df_rate=0.1)
    
    #Создаем модель
    model = artm.ARTM(num_topics=select_topics, dictionary=dictionary, show_progress_bars=False)
    

    # scores
    model.scores.add(artm.PerplexityScore(name='PerplexityScore', dictionary=dictionary))
    model.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
    model.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))

    # 1st regularizer
    model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=-1.0)) #-0.1
    model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=5)

    # 2st regularizer
    model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.5))
    model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)

    # 3st regularizer
    model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=1.5e+5))

    model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)
    
    print (f'Модель для {file}')
    print(f"\tSparsityThetaScore: {model.score_tracker['SparsityThetaScore'].last_value}")
    print(f"\tSparsityPhiScore: {model.score_tracker['SparsityPhiScore'].last_value}")
    print(f"\tPerplexityScore: {model.score_tracker['PerplexityScore'].last_value}")
    
    # Сохраняем модель в каталог  PATH_BIGARTM_MODEL 
    file_mod = os.path.splitext(os.path.splitext(file)[0])[0]          
    model.dump_artm_model(PATH_BIGARTM_MODEL+file_mod)
        
    

Модель для Internet_i_SMI.csv.bz2
	SparsityThetaScore: 0.5631956458091736
	SparsityPhiScore: 0.8461095690727234
	PerplexityScore: 646.8890380859375
Модель для Biznes.csv.bz2
	SparsityThetaScore: 0.5861086249351501
	SparsityPhiScore: 0.8355470299720764
	PerplexityScore: 252.99160766601562
Модель для Sport.csv.bz2
	SparsityThetaScore: 0.6888564825057983
	SparsityPhiScore: 0.8592645525932312
	PerplexityScore: 328.2021179199219


In [14]:
model=artm.load_artm_model('/home/midzay/Mlerning/GitHUb/topic_news/bigartm/data/model/bigartm/Internet_i_SMI/')

In [15]:
phi = model.get_phi()
phi['word'] = phi.index
phi['word']=phi.word.apply(lambda x: x[1])

for col in phi.columns:
    if col != 'word':
        print(f"{col}: {phi[[col, 'word']].sort_values(by=col, ascending=False)['word'].values[:15]}")

topic_0: ['google' 'приложение' 'яндекс' 'мобильный' 'устройство' 'apple' 'запрос'
 'поисковый' 'microsoft' 'поисковик' 'поиск' 'позволять' 'yahoo'
 'компьютер' 'помощь']
topic_1: ['холдинг' 'иск' 'эхо' 'радиостанция' 'редактор' 'гендиректор' 'акция'
 'генеральный' 'должность' 'медиа' 'пост' 'дом' 'акционер' 'коммерсант'
 'совет']
topic_2: ['британский' 'news' 'великобритания' 'бибись' 'bbc' 'daily' 'житель'
 'ребёнок' 'лондон' 'корпорация' 'guardian' 'world' 'фунт' 'возраст'
 'таблоид']
topic_3: ['реклама' 'миллиард' 'доллар' 'рынок' 'рубль' 'составить' 'рекламный'
 'доля' 'продажа' 'аудитория' 'составлять' 'стоимость' 'месяц' 'рост'
 'доход']
topic_4: ['роскомнадзор' 'запретить' 'блокировка' 'ведомство' 'требование'
 'telegram' 'законопроект' 'нарушение' 'госдума' 'реестр' 'заблокировать'
 'законодательство' 'документ' 'мессенджер' 'депутат']
topic_5: ['украина' 'украинский' 'корреспондент' 'задержать' 'убийство' 'произойти'
 'журналистка' 'нападение' 'акция' 'здание' 'киев' 'смерть'

In [16]:
# Текст из рубрики  интернет и сми
text =['москва', 'отель', 'аэростарый', 'открыться', 'московский', 'международный', 'выставка',
       'шоу', 'посвятить', 'новый', 'достижение', 'область', 
       'технология', 'выставка', 'продемонстрировать', 'последний', 'разработка',
       'существующий', 'рынок',  'оборудование', 'программный', 'обеспечение', 'выставка', 'продлиться']

In [19]:
# Приводим к формату  vowpal_wabbit
vwpath= f'data/vwpath/999_input_bigartm.vw'
with open(vwpath, 'w') as fp:
    fp.write('| {}\n'.format(' '.join(text)))
    
    # Создаем batches 
batches_path = f'data/batches/999'
if not os.path.exists(batches_path):
    os.makedirs(batches_path)
    
bv = artm.BatchVectorizer(data_path=vwpath,data_format='vowpal_wabbit',target_folder=batches_path)

In [20]:
theta = model.transform(batch_vectorizer=bv)
theta.T

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,0.106827,0.0,0.0,0.119627,0.0,0.0,0.068265,0.0,0.371066,0.0,0.0,0.0,0.0,0.334215,0.0
